# Byggingagreining á gervitunglamyndum
Nathan HK

In [162]:
from io import BytesIO
import math
import matplotlib.pyplot as plt
from PIL import Image
from pyrosm import OSM
from pyrosm import get_data
import pywikibot
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from shapely.geometry import Point
import time

## Inngangsorð
Við viljum þjálfa gervigreindarlíkan til að greina byggingar á gervitunglamyndum. Við þurfum tvenn gögn: gervitunglamyndir og staðsetningar bygginga.
- **Gervitunglamyndir:** Við notum skjámyndatökur af Já.is. Ég veit ekki hvort þetta sé löglegt, en ég er ekki með neinar betri leiðir.
- **Staðsetningar bygginga:** Við notum OpenStreetMap.

## Gervitunglamyndir
Við notum lista yfir greinar á ensku Wikipediunni um staði á Höfuðborgarsvæðinu og Akureyri. Gögnin á Landsbyggðinni eru ekki nóg nákvæm fyrir þetta líkan.

In [163]:
byrjun = time.time()
hnitlisti = []
wiki = pywikibot.Site('en', 'wikipedia')

# Capital Region
flokkur_h = pywikibot.Category(wiki, 'Populated places in Capital Region (Iceland)')
undir = list(flokkur_h.subcategories(recurse=3))
buinn_fl = []  # Completed categories
buinn_si = []  # Completed pages
print(len(undir))
for a in undir:
    if a in buinn_fl:  # Already did category
        continue
    for b in a.articles():
        if b in buinn_si:  # Already did page
            continue
        coord = b.coordinates(primary_only=True)
        # Coordinates are invalid outside latitude [63, 67] and longitude [-25, -12]
        if coord is not None and coord.lat >= 63 and coord.lat <= 67 and coord.lon >= -25 and coord.lon <= -12:
            hnitlisti.append(('h', float(coord.lat), float(coord.lon)))
        buinn_si.append(b)
    buinn_fl.append(a)
print('h', len(hnitlisti), time.time() - byrjun)

# Akureyri
flokkur_a = pywikibot.Category(wiki, 'Akureyri')
undir = list(flokkur_a.subcategories(recurse=3))
buinn_fl = []  # Completed categories
buinn_si = []  # Completed pages
print(len(undir))
for a in undir:
    if a in buinn_fl:  # Already did category
        continue
    for b in a.articles():
        if b in buinn_si:  # Already did page
            continue
        coord = b.coordinates(primary_only=True)
        # Coordinates are invalid outside latitude [63, 67] and longitude [-25, -12]
        if coord is not None and coord.lat >= 63 and coord.lat <= 67 and coord.lon >= -25 and coord.lon <= -12:
            hnitlisti.append(('a', float(coord.lat), float(coord.lon)))
        buinn_si.append(b)
    buinn_fl.append(a)
print('a', len(hnitlisti), time.time() - byrjun)

54
h 120 93.18046808242798
6
a 130 108.11664319038391


Við getum ekki tekið myndir af miðjunni skjásins, því það eru önnur HTML-efni sem hylja gervitunglamyndirnar. Þess vegna leitum við að staði sem eru 0.002° til austurs frá myndatökustaðnum; hérna eru hnitin á skjánum fyrir þennan stað.

In [164]:
skhn = (746, 861)

Við tökum skjámyndir af öllum stöðum á hnitlistanum.
- URL-ið notar ISN93-hnit, en okkar hnit eru WGS84, og það er engin einföld leið til að skipta milli þeirra. Þess vegna þurfum við að leita að hnitum eins og manneskja myndi leita.
- Á Chrome er myndasvæðið 512x512, en þegar myndin er vistuð verður hún 1024x1024.

In [165]:
byrjun = time.time()
driver = webdriver.Chrome()
driver.set_window_size(1500, 1000)
driver.get('https://ja.is/kort/?x=356954&y=408253&nz=17.00&type=aerialnl')
# Close GDPR banner
btn = driver.find_element(By.XPATH, '//a[@id="gdpr_banner_ok"]')
btn.click()
leit = driver.find_element(By.XPATH, '//input[@id="mapq"]')
for n in range(len(hnitlisti)):
    if n % 50 == 0:
        print(n, time.time() - byrjun)
    hnit = hnitlisti[n]
    # Input search term into search box
    leit.clear()
    leit.send_keys(str(hnit[1]) + ', ' + str(hnit[2] + 0.002))
    leit.send_keys(Keys.RETURN)
    time.sleep(2) # Wait for images to load
    try:  # Place not found
        nf = driver.find_element(By.XPATH, '//div[@class="row not-found"]')
    except NoSuchElementException:  # Place found, save and crop screenshot
        driver.save_screenshot('/Users/002-nathan/Desktop/Envalys/gtm/' + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
        skmynd = Image.open('/Users/002-nathan/Desktop/Envalys/gtm/' + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
        skmynd = skmynd.crop((skhn[0] - 512, skhn[1] - 512, skhn[0] + 512, skhn[1] + 512))
        skmynd.save('/Users/002-nathan/Desktop/Envalys/gtm/' + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
    time.sleep(1)
driver.close()
print(time.time() - byrjun)

0 3.9994421005249023
50 217.97834610939026
100 430.97044920921326
557.2064530849457


## Staðsetningar bygginga
Við sækjum gögn frá OpenStreetMap.

In [166]:
fp = get_data('Iceland')

Þetta undirforrit tekur díl á myndinni og finnur GPS-hnitin. Við notum Web Mercator.

In [167]:
def pix2coord(pix, hnit_br):
    x_t = hnit_br[1] + (pix[0] - 512) / 1024
    lon = math.degrees(x_t * 2 * math.pi / (2 ** 17) - math.pi)
    y_t = hnit_br[0] + (pix[1] - 512) / 1024
    lat = math.degrees(2 * (math.atan(math.exp(math.pi - y_t * 2 * math.pi / (2 ** 17))) - math.pi / 4))
    return (lat, lon)

Við sækjum tvo lista: einn yfir byggingar á Höfuðborgarsvæðinu, og einn á Akureyri.

In [168]:
byrjun = time.time()
byg_listi = {}
osm_h = OSM(fp, bounding_box=[-22.140901, 63.847886, -21.152576, 64.390306])
byg_listi['h'] = osm_h.get_buildings()
osm_a = OSM(fp, bounding_box=[-18.398071, 65.543087, -17.968359, 66.576398])
byg_listi['a'] = osm_a.get_buildings()
print(time.time() - byrjun)

32.90095901489258


Fyrir hverja skjámynd búum við til nýja mynd sem sýnir staðsetningar bygginga. Dílar með byggingum eru rauðir, og dílar án bygginga eru bláir.

Til að gera þetta fljótari:
- Við útreiknum utanferninginn (e. *bounding box*) fyrir hverja byggingu áður en við skoðum myndina. Ef díll er ekki í utanferningnum, þá er hann ekki í byggingunni, og við vitum það án þess að gera ```.contains()```, sem tekur langan tíma.
- Við búum til fjögur ílát: eitt fyrir byggingar sem eru norðvestur af miðpunktnum, eitt fyrir norðaustur, o.s.fv. Fyrir hverja mynd setjum við byggingarnar í viðeigandi ílátið, og fyrir hvern díl þurfum við bara að leita þar.

In [187]:
byrjun = time.time()
bd_all = {}
for st in ['h', 'a']:
    byggingar = byg_listi[st]
    bd = []
    for k in range(byggingar.shape[0]):
        bns = byggingar['geometry'][k].bounds
        bd.append(bns)
    bd_all[st] = bd
for n in range(len(hnitlisti)):
    if n % 10 == 0:
        print(n, time.time() - byrjun)
    hnit = hnitlisti[n]
    # Open image
    try:
        gtm = Image.open('/Users/002-nathan/Desktop/Envalys/gtm/' + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
        dilar = gtm.load()
    except FileNotFoundError:
        continue
    # Convert coordinates
    y_n = 1 / (2 * math.pi) * 2 ** 17 * (math.pi - math.log(math.tan(math.pi / 4 + math.radians(hnit[1]) / 2))) - 0.5
    y_s = 1 / (2 * math.pi) * 2 ** 17 * (math.pi - math.log(math.tan(math.pi / 4 + math.radians(hnit[1]) / 2))) + 0.5
    byggingar = byg_listi[hnit[0]]
    hnit_br = (1 / (2 * math.pi) * 2 ** 17 * (math.pi - math.log(math.tan(math.pi / 4 + math.radians(hnit[1]) / 2))),
               1 / (2 * math.pi) * 2 ** 17 * (math.pi + math.radians(hnit[2])))
    # Buildings
    bd = bd_all[hnit[0]]
    ilat = [[[], []], [[], []]]
    for k in range(byggingar.shape[0]):
        bns = bd[k]
        if bns[0] <= hnit[2] and bns[1] <= hnit[1] and bns[2] >= hnit[2] - 0.0014 and bns[3] >= hnit[1] - 0.0008:
            ilat[0][0].append(k)
        if bns[0] <= hnit[2] and bns[3] >= hnit[1] and bns[2] >= hnit[2] - 0.0014 and bns[1] <= hnit[1] + 0.0008:
            ilat[0][1].append(k)
        if bns[2] >= hnit[2] and bns[1] <= hnit[1] and bns[0] <= hnit[2] + 0.0014 and bns[3] >= hnit[1] - 0.0008:
            ilat[1][0].append(k)
        if bns[2] >= hnit[2] and bns[3] >= hnit[1] and bns[0] <= hnit[2] + 0.0014 and bns[1] <= hnit[1] + 0.0008:
            ilat[1][1].append(k)
    # Loop thru pixels
    for i in range(1024):
        for j in range(1024):
            coord = pix2coord((i, j), hnit_br)
            f = True
            ilat_pos = [1, 1]
            if coord[1] < hnit[2]:
                ilat_pos[0] = 0
            if coord[0] < hnit[1]:
                ilat_pos[1] = 0
            for k in ilat[ilat_pos[0]][ilat_pos[1]]:
                if bd[k][0] > coord[1] or bd[k][1] > coord[0] or bd[k][2] < coord[1] or bd[k][3] < coord[0]:
                    continue
                if byggingar['geometry'][k].contains(Point(coord[1], coord[0])):
                    gtm.putpixel((i, j), (max(dilar[i,j][0] + 64, 255), dilar[i,j][1], dilar[i,j][2]))
                    f = False
                    break
            if f:
                gtm.putpixel((i, j), (dilar[i,j][0], dilar[i,j][1], max(dilar[i,j][2] + 64, 255)))
    gtm.save('/Users/002-nathan/Desktop/Envalys/gtm/saman_' + hnit[0] + '_' + str(hnit[1]) + '_' + str(hnit[2]) + '.png')
print(time.time() - byrjun)

0 0.512286901473999
10 90.64847302436829
20 180.5462188720703
30 290.40744495391846
40 423.5607581138611
50 506.9982409477234
60 604.9740378856659
70 672.0114297866821
80 732.5899751186371
90 860.6018071174622
100 1018.5698711872101
110 1101.3274369239807
120 1210.532191991806
1277.1505191326141


## Lokaorð
Við erum með forrit sem sækir gervitunglamyndir og kort af staðsetningum bygginga á mjög nákvæman hátt. Þetta er allt sem við þurfum til að þjálfa gervigreindarlíkan.